<a href="https://colab.research.google.com/github/sshekhar17/GaussianProcess/blob/master/Pytorch_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from torch import nn, optim 
import torch.nn.functional as F 
from torchvision import transforms
from math import sqrt
import numpy as np 
import fnmatch 
from random import shuffle
from PIL import Image
import matplotlib.pyplot as plt


In [2]:
from google.colab import files 
files.upload()

!mkdir ./UTKData
!tar -C ./UTKData -xvzf crop_part1.tar.gz
# !cd UTKData/crop_part1/

Streaming output truncated to the last 5000 lines.
crop_part1/14_0_0_20170110224441502.jpg.chip.jpg
crop_part1/62_0_2_20170104212649180.jpg.chip.jpg
crop_part1/24_1_3_20170104222905601.jpg.chip.jpg
crop_part1/43_0_1_20170104185752431.jpg.chip.jpg
crop_part1/18_1_4_20170103222943622.jpg.chip.jpg
crop_part1/29_1_4_20170103230119816.jpg.chip.jpg
crop_part1/1_0_2_20161219212648582.jpg.chip.jpg
crop_part1/2_0_2_20161219222111287.jpg.chip.jpg
crop_part1/65_0_0_20170111203730182.jpg.chip.jpg
crop_part1/24_1_3_20170104222402975.jpg.chip.jpg
crop_part1/13_0_0_20170110225438328.jpg.chip.jpg
crop_part1/24_0_0_20170104165239658.jpg.chip.jpg
crop_part1/47_1_0_20170109132641773.jpg.chip.jpg
crop_part1/2_1_2_20161219162336446.jpg.chip.jpg
crop_part1/21_0_4_20161223214827816.jpg.chip.jpg
crop_part1/60_0_0_20170109013457906.jpg.chip.jpg
crop_part1/68_1_0_20170110175345439.jpg.chip.jpg
crop_part1/52_1_0_20170110143721250.jpg.chip.jpg
crop_part1/28_1_1_20170103225945785.jpg.chip.jpg
crop_part1/5_1_1_2016

In [3]:
data_dir = './UTKData/crop_part1/' 
# data_dir = './UTKData/UTKFace'



temp = os.listdir(data_dir)
dataset = fnmatch.filter(temp, '*_*_*_*')
m = len(dataset)
shuffle(dataset)

age = [i.split('_')[0] for i in dataset]
gender = [i.split('_')[1] for i in dataset]
ethnicity = [i.split('_')[2] for i in dataset]


#Since ages are in strings we need to type cast to integer.
age = list(map(int, age))
gender = list(map(int,gender))
ethnicity = list(map(int,ethnicity))

# Create a dict decoding the ethnicities
eth_dict = {
    0:'White',
    1:'Black',
    2:'Asian',
    3:'Indian', 
    4:'Others'
}

# Dict decoding the genders
gen_dict = {
    1:'Female',
    0:'Male'
}

# Dset = []
# for i, filename in enumerate(dataset):
#   if i%1000==999:
#     print('Completed {}'.format(i+1))
#   im = Image.open(data_dir+ '/' + filename)
#   ### For CNN COMMENT THE LINE BELOW 
#   # tensor_im = transforms.ToTensor()(im).view(3,-1)
#   tensor_im = transforms.ToTensor()(im)

#   Dset.append(tensor_im)
# # Stack them to construct a tensor dataset of size 9778 x 3 x 40000
# Dset = torch.stack(Dset)





# print('-'*20 + '\n')
# # Print the fraction of samples from each group
# for eth in eth_dict:
#   n_eth = len([1  for ei in ethnicity if ei==eth])
#   print("The fraction of {} samples is {:.3f}".format(eth_dict[eth], n_eth/len(ethnicity) ))
# # Print the gender fractions
# print('-'*20 + '\n')
# for gen in gen_dict:
#   n_gen = len([1 for g in gender if g==gen]) 
#   print("The fraction of {} samples is {:.3f}".format(gen_dict[gen], n_gen/len(gender) ))

In [47]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform

class UTKDataset(Dataset):
  def __init__(self, root_dir, transform=None, group='all'):
    """
    Args:

    group   :str  {'all', '0', '1', '2', '3', '4'}
    """
    self.root_dir = root_dir 
    self.transform = transform 


    temp = os.listdir(data_dir)
    dset = fnmatch.filter(temp, '*_*_*_*')
    # save the list of filenames
    self.dataset = dset
    # length of the dataset
    self.length = len(dset)

    # shuffle(dset)
    
    age = [i.split('_')[0] for i in dset]
    gender = [i.split('_')[1] for i in dset]
    ethnicity = [i.split('_')[2] for i in dset]
    
    #Since ages are in strings we need to type cast to integer.
    self.age = list(map(int, age))
    self.gender = list(map(int,gender))
    self.ethnicity = list(map(int,ethnicity))
    
  
    
    # Create a map decoding the ethnicities
    self.eth_dict = {
        0:'White',
        1:'Black',
        2:'Asian',
        3:'Indian', 
        4:'Others'
    }
    # Create the map decoding the gender
    self.gen_dict = {
        0: 'Male', 
        1: 'Female'
    }
    #
    ###Subsample the dataset based on the input 'group'
    self.subsample(group)
    self.transform = transform
    

  def subsample(self, group):
    '''
    based on the value of the input 'group', this method 
    subsamples the dataset and modifies the other attributes 
    accordingly. 

    This will be used in creating the oracle corresponding 
    to the different protected groups.
    '''
    if group=='all':
      self.group = 'all'
      return 
    else:
      g = int(group)
      self.group = self.eth_dict[g]
      Idx = [i for i in range(len(self.dataset)) if self.ethnicity[i]==g]
      # Obtain the new dataset 
      new_dataset = [self.dataset[i] for i in Idx]
      new_age = [self.age[i] for i in Idx]
      new_gender = [self.gender[i] for i in Idx]
      new_ethnicity = [self.ethnicity[i] for i in Idx]
      new_length = len(Idx)

      # Update the attributes. 
      self.dataset = new_dataset 
      self.age = new_age 
      self.gender = new_gender 
      self.ethnicity = new_ethnicity 
      self.length = new_length

  def __len__(self):
    return self.length

  def __getitem__(self, idx):
    '''
    idx is either an integer or a tensor of integers
    '''
    if torch.is_tensor(idx):
      idx = idx.tolist()
      # get a list of image names 
      image_names = [os.path.join(self.root_dir,
                          self.dataset[i]) for i in idx]
      # read a list of (numpy) images
      images = [io.imread(img_name) for img_name in image_names]
      # list of labels
      labels = [self.gender[i] for i in idx]
      # Construct a list of dicts 
      samples_dict  = [
                        {'image':img, 'label':lbl} 
                        for img, lbl in zip(images, labels)
      ]
      if self.transform is not None:
        # assuming that transforms also include ToTensor()
        transformed_samples = [
                                self.transform(s) for s in samples_dict
        ]
        X = torch.stack([s['image'] for s in transformed_samples])
        Y = torch.stack([s['label'] for s in transformed_samples])
        # return a batch of data
        return X, Y

      else:
        transformed_samples = samples_dict
      # return a list of dicts
      return transformed_samples
    else: 
      img_name = os.path.join(self.root_dir,
                              self.dataset[idx])
      image = io.imread(img_name)
      label= self.gender[idx]
      sample = {'image': image, 'label': label}

      if self.transform:
          sample = self.transform(sample)
      return sample


#### Test the class
gp = torch.randint(0, 5, (1,)).item()
mydataset = UTKDataset(root_dir = data_dir, group=str(gp))

# i = np.random.randint(0, len(mydataset))
# im = mydataset[i]
# # print('There are {} elements in group: {}'.format(len(mydataset), mydataset.group))

# print('Gender is {} from Group {}'.format(mydataset.gen_dict[im['label']], 
#                                           gp))
# plt.imshow(im['image'])


idx = torch.randint(0, 1000, (3,))
sample = mydataset[idx]
a = sample[0]
print(type(a['image']))

<class 'numpy.ndarray'>


In [46]:
a = [torch.randint(0, 10, (2,5)) for _ in range(10)]

b = torch.stack(a)
print(b.shape)

torch.Size([10, 2, 5])


In [42]:
from skimage.transform import AffineTransform, warp

class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (int): Desired output size. 
    """
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size

    def __call__(self, sample):
        image = sample['image']
        new_h, new_w = self.output_size, self.output_size
        img = transform.resize(image, (new_h, new_w))

        return {'image': img, 'label': sample['label']}


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

### RotateImage modified from this link: https://medium.com/jdsc-tech-blog/multioutput-cnn-in-pytorch-c5f702d4915f

class RotateImage(object):
    def __call__(self, sample):
        image = sample['image']

        min_scale = 0.8
        max_scale = 1.2
        sx = np.random.uniform(min_scale, max_scale)
        sy = np.random.uniform(min_scale, max_scale)
        # --- rotation ---
        max_rot_angle = 7
        rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.
        # --- shear ---
        max_shear_angle = 10
        shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.
        # --- translation ---
        max_translation = 4
        tx = np.random.randint(-max_translation, max_translation)
        ty = np.random.randint(-max_translation, max_translation)
        tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
        transformed_image = warp(image, tform)
        assert transformed_image.ndim == 3
        return {'image': transformed_image, 'label': sample['label']}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __init__(self, normalize=True):
      self.normalize=normalize

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        image = torch.from_numpy(image)
        if self.normalize:
          N = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])
          image = N(image)
        return {'image': image,
                'label': torch.Tensor([label])}




In [51]:
# Define the composition of the transforms
ComposedTransforms = transforms.Compose([RotateImage(), Rescale(50),
                                        ToTensor() 
                                        # transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        # std=[0.229, 0.224, 0.225])                             
                               ])

# sample = mydataset[i]
# trnsf_sample = ComposedTransforms(sample)
# image = trnsf_sample['image']
# image = image.permute(1, 2, 0)
# print(image.shape)
# plt.imshow(image.numpy())

dset0 = UTKDataset(root_dir=data_dir, transform=ComposedTransforms, group='0')
m0 = len(dset0)
# dataloader = DataLoader(dset0, batch_size=40,
#                         shuffle=True, num_workers=0)

# for i, b in enumerate(dataloader):
#   print(b['label'].sum())
#   break 
idx = torch.randint(0, m0, (4,))

torch.Size([4, 1])

In [ ]:
def categorize_age(a):
  '''
Categorize people into four groups according to their age.
  '''
  if a<=7:
    return 0 
  elif 8<=a<=25:
    return 1 
  elif 26<=a<=50:
    return 2 
  else:
    return 3 


class TrainingOracle(
    
):
def __init__(self, root_dir, batch_size=40, tx=None):
  dset0 = UTKDataset(root_dir=root_dir, transform=tx, group='0')
  dset1 = UTKDataset(root_dir=root_dir, transform=tx, group='1')
  dset2 = UTKDataset(root_dir=root_dir, transform=tx, group='2')
  dset3 = UTKDataset(root_dir=root_dir, transform=tx, group='3')
  dset4 = UTKDataset(root_dir=root_dir, transform=tx, group='4')

  pass 

####_______________________________________________________________________________
#### TODO: 
#### 1. vectorize the getitem of UTKDataset class.
#### 2. Use this in the TrainingOracle Definition.
####_______________________________________________________________________________




# Defines the Oracle for the UTKFace dataset: This oracle takes in as inputs the entire tensor 
# dataset and then returns a uniformly drawn (with replacement) sample from the dataset corresponding 
# to a given ethnicity (att). 
def Oracle(Dset, ethnicity, att=0, batch_size=20, 
           add_noise=True, noise_amp=0.1, device='cpu',
           label_age=False):
  # get the list of indices for the attribute att
  Idx = [i for i,e in enumerate(ethnicity) if e==att] 
  # Draw the random samples with replacement
  random_ints = torch.randint(low=0, high=len(Idx), size=(batch_size,))
  batch_x = [Dset[Idx[i]] for i in random_ints] 
  batch_x = torch.stack(batch_x) # batch_size x 3 x 40000 size tensor 
  # Obtain the corresponding labels
  if label_age:
    batch_y = torch.Tensor([
                            categorize_age(age[Idx[i]]) for i in random_ints
    ])
  else:
    batch_y = torch.Tensor([gender[Idx[i]] for i in random_ints])
  # Need to convert to Long int for F.cross_entropy
  batch_y = batch_y.long()
  # Add some noise to the batch if required
  if add_noise:
    ## this version causes errors due to negative values
    # batch_x = batch_x + (torch.rand(batch_x.shape)*2.0-1.0)*noise_amp
    ## Use this instead
    batch_x = batch_x + torch.rand(batch_x.shape)*noise_amp
  # Send the batch of data to the appropriate device 
  return batch_x.to(device), batch_y.to(device)

X, Y = Oracle(Dset, ethnicity, att=1, batch_size=10, label_age=True)
print(X.shape, Y.shape)
print(Y)

torch.Size([10, 3, 200, 200]) torch.Size([10])
tensor([2, 3, 2, 2, 1, 0, 2, 2, 2, 2])


In [ ]:
# Convolutional Neural Network Module 

# Define a function which calculates the output size 
def outputSize(in_size, kernel_size, stride, padding):
  output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
  return(output)

class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 200, 200)
    def __init__(self, num_labels=2):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 10
        self.conv1 = torch.nn.Conv2d(3, 10, kernel_size=10, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=10, stride=2, padding=0)
        
        # calculate the size of the output from the conv2d layer
        self.output_size1 = outputSize(193, kernel_size=10, stride=2, padding=0)
        
        #output_size1*output_size1*10 input features, num_labels output features 
        self.fc1 = torch.nn.Linear(10 * self.output_size1 * self.output_size1, num_labels)
        
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 200, 200) to (10, 200, 200)
        x = F.relu(self.conv1(x))

        #Size changes from (10, 193, 193) to (10, 92, 92)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 10 * self.output_size1 *self.output_size1)

        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 2)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(x, dim=1)
        return(x)


In [ ]:
# Define the Linear Model
class LinReg(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.lin = nn.Linear(input_size, output_size)

  def forward(self, xb):
    if len(xb.shape)==3:
      xb = xb.view(xb.shape[0], -1)
      # print(xb.device)
    temp = self.lin(xb)
    # print('Shape of temp is {}'.format(temp.shape))
    output = F.log_softmax(temp, dim=1)
    # print('Shape of output is {}'.format(output.shape))
    return output

# Define the loss function:
# loss_func = F.binary_cross_entropy
loss_func = F.nll_loss

# Function to initialize the model
def get_model(device, input_size=200*200*3,  lr=1e-4, cnn=True, age=False):
  output_size=4 if age else 2
  if cnn:
    model= SimpleCNN(num_labels=output_size)
  else:
    model = LinReg(input_size=input_size, output_size=output_size)
  model.to(device)
  return model, optim.SGD(model.parameters(), lr=lr)


In [ ]:
# Check if gpu is available 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model Initialization (with default parameters)
model, opt = get_model(device=device, age=False, cnn=True)

# Log softmax 
# def log_softmax(x):
    # return x - x.exp().sum(-1).log().unsqueeze(-1)


print('-'*50)

def getEmpiricalLoss(model, counts, device, bs=20, ucb_factor=0.0):
  Correct = [0]*len(counts)
  model.eval()
  with torch.no_grad():
    for att in range(len(counts)):
      #validation batch_size
      vbs = int(counts[att]*bs)
      #get the samples 
      xv, yv = Oracle(Dset, ethnicity, att=att, batch_size=vbs, device=device)
      #compute the output 
      output = model(xv)
      pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
      #Compute the empirical loss  +
      Correct[att] = pred.eq(yv.view_as(pred)).sum().item()/vbs - ucb_factor/sqrt(vbs)
  return torch.Tensor(Correct)

--------------------------------------------------


In [ ]:
# List of Counts of 
counts = torch.Tensor([0]*5)
### Trial Training Run
### Adaptive sampling scheme
# scheme = 'ucb'
scheme = 'epsilon-greedy'
# scheme = 'uniform'
# scheme='empirical'


if scheme=='ucb':
  ucb_factor = 0.1
elif scheme=='epsilon-greedy':
  epsilon=0.1
  ucb_factor=0.0
elif scheme=='uniform':
  pass 
elif scheme=='empirical':
  scheme='epsilon-greedy'
  epsilon=0.0 
  ucb_factor=0.0

for i in range(500):
  #######################################
  #Choose the attribute:
  if i<5:
    att = i 
    counts[i]+=1
  #Method~1: Randomly choose attribute == Uniform mixture.  
  else:
    if scheme=='uniform':
      # att = torch.randint(low=0, high=5, size=(1,)).item()
      att = i%5
      counts[att]+=1
    elif scheme=='ucb':
    #Method~1: UCB
      LCB = getEmpiricalLoss(model, counts, device,ucb_factor = ucb_factor)
      att = LCB.argmin().item()
      counts[att]+=1
    elif scheme=='epsilon-greedy':
    #Method~2: epsilon-greedy
      if torch.rand((1,)).item()< epsilon:
        att = torch.randint(low=0, high=5, size=(1,)).item()
      else:
        LCB = getEmpiricalLoss(model, counts, device,ucb_factor = 0.0)
        att = LCB.argmin().item()
      counts[att]+=1

      




  #######################################
  xb, yb = Oracle(Dset, ethnicity, att=att, device=device, label_age=False)
  # predict
  pred = model(xb)
  # print(pred.shape, pred.dtype, yb.dtype)
  # break 
  loss = loss_func(pred, yb)

  loss.backward()
  opt.step()
  opt.zero_grad()
  if i%10==9:
    print('Completed {} iterations'.format(i+1))

Completed 10 iterations
Completed 20 iterations
Completed 30 iterations
Completed 40 iterations
Completed 50 iterations
Completed 60 iterations
Completed 70 iterations
Completed 80 iterations
Completed 90 iterations
Completed 100 iterations
Completed 110 iterations
Completed 120 iterations
Completed 130 iterations
Completed 140 iterations
Completed 150 iterations
Completed 160 iterations
Completed 170 iterations
Completed 180 iterations
Completed 190 iterations
Completed 200 iterations


RuntimeError: ignored

In [ ]:
model.eval()
test_loss = [] 
correct = []
test_batch_size=1000
with torch.no_grad():
  for att in range(5):
    data, target = Oracle(Dset, ethnicity, att=att, batch_size=test_batch_size, device=device)
    output = model(data)
    test_loss1 = F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct1 = pred.eq(target.view_as(pred)).sum().item()
    test_loss.append(test_loss1)
    correct.append(correct1/test_batch_size)


print('Accuracy over the five groups is {}'.format(correct))
print('Minmax accuracy is {:.3f}'.format(
  torch.Tensor(correct).min().item()
))
print('Count of Queries = {}'.format(counts))
